In [50]:
import pandas as pd
import numpy as np

dati = pd.read_excel("Interferometro.xlsx")

# distanze in metri
display(dati)
L_focale = 18 * 10**(-3)
distanza_lente_schermo = 1.605

distanza_eff = distanza_lente_schermo - L_focale

raggi = dati["raggio"] * 10**(-2)

angoli = np.arctan(raggi/distanza_eff)

coseni = np.cos(angoli)

from iminuit import Minuit
from iminuit.cost import LeastSquares

def linear_cost(x, a, b):
    return a + b*x

x = [4,3,2,1,0]

minimi_quadrati = LeastSquares(x, coseni[:5], 10**(-5), linear_cost)

m = Minuit(minimi_quadrati, a=0, b=0)

m.migrad()
m.hesse()
display(m)
d = -1/(2*m.values["b"])*(630*10**(-9))
print(d)

N_0 = m.values["a"]*2*d/(630*10**(-9))
print(N_0)



,L focale [mm],18,d [micrometri],n frange,raggio,costheta,x,y,Unnamed: 8,Unnamed: 9,Deltad eff [microm],DeltaN,DeltaN medio,Unnamed: 13,Michelson,Unnamed: 15,Deltad eff [microm].1,DeltaN.1
0,"d(lente,schermo) [cm]",160.5000,26.0,5.0,3.70,0.999734,0.0,0.000000,NaN,NaN,10.0,27.0,29.4,NaN,"d(lente,schermo) [cm]",167.8,10.0,29.0
1,lamda [microm],0.6328,NaN,NaN,3.30,0.999789,1.0,0.000054,NaN,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN,33.0
2,NaN,NaN,NaN,NaN,2.90,0.999837,2.0,0.000102,NaN,NaN,NaN,29.0,NaN,NaN,NaN,NaN,NaN,28.0
3,NaN,NaN,NaN,NaN,2.20,0.999906,3.0,0.000172,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,29.0
4,NaN,NaN,NaN,NaN,1.35,0.999965,4.0,0.000230,NaN,NaN,NaN,29.0,NaN,NaN,NaN,NaN,NaN,31.0
5,NaN,NaN,46.0,NaN,3.70,0.999734,NaN,0.000000,NaN,NaN,7.0,19.0,20.4,NaN,NaN,NaN,7.0,22.0
6,NaN,NaN,NaN,NaN,3.30,0.999789,NaN,0.000054,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,21.0
7,NaN,NaN,NaN,NaN,2.90,0.999837,NaN,0.000102,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,18.0
8,NaN,NaN,NaN,NaN,2.25,0.999902,NaN,0.000167,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,21.0
9,NaN,NaN,NaN,NaN,1.40,0.999962,NaN,0.000228,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,20.0


┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.203 (χ²/ndof = 0.4)      │              Nfcn = 54               │
│ EDM = 8.52e-16 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ a    │999.961e-3 │ 0.008e-3  │            │            │         │         │       │
│ 1 │ b    │ -59.1e-6  │  3.2e-6   │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬───────────────────┐
│   │        a        b │
├───┼───────────────────┤
│ a │    6e-11 -0.02e-9 │
│ b │ -0.02e-9    1e-11 │
└───┴───────────────────┘

0.005329640867532765
16918.831809114392


In [62]:
delta_N = dati["DeltaN"]

medie_delta_N = []
sigma_delta_N = []

# Calcola la media di ciascun set
for i in range(0, len(delta_N), 5):
    current_set = delta_N[i:i+5]  # Ottieni il set corrente

    media_set = np.mean(current_set)  # Calcola la media
    medie_delta_N.append(media_set)  # Aggiungi la media all'array medie_delta_N  = []

    err_set = np.std(current_set) / np.sqrt(len(current_set))
    sigma_delta_N.append(err_set)

_lambda = 632.8 * 10**(-9)

delta_d = medie_delta_N[2]*_lambda/2 # costheta circa 1 (non abbiamo calcolato il raggio)
sigma_delta_d = sigma_delta_N[2]*_lambda/2

display(delta_d)
display(sigma_delta_d)


4.61944e-06

1.1319870529294936e-07